In [ ]:
from vpython import *

# Function that facilitates the input of a nonnegative integer
def inputNumber(inputQuestion, inputQuery):
    inputAnswer = input(inputQuestion + " " + inputQuery) # Asks for user input
    while inputAnswer.isdigit() == False: # Keeps asking for input until answer is a digit
        inputAnswer = input("Sorry, your input is not a nonnegative integer. " + inputQuery)
    return int(inputAnswer)

# Function that facilitates the input of a nonnegative integer in a range
def inputNumberRange(inputQuestion, minim, maxim):
    inputAnswer = inputNumber(inputQuestion, "Please enter a nonnegative integer between {} and {}: "
                              .format(minim, maxim)) # Asks for user input
    while inputAnswer < minim or inputAnswer > maxim: # Keeps asking for input until answer is between minim & maxim
        inputAnswer = inputNumber("Sorry, your input is not between {} and {}.".format(minim, maxim),
                                  "Please enter a nonnegative integer between {} and {}: ".format(minim, maxim))
    return inputAnswer
    

input("Welcome to the Newton's Cradle Simulator! Press enter to continue:")
print()

length = 10 # Sets length of strings

# Creates dictionaries of balls & strings
ball = {}
string = {}

# Sets min & max numbers of balls
minBalls = 3
maxBalls = 8

# Asks for desired number of balls
ballNumber = inputNumberRange("How many balls would you like in your Newton's cradle?", minBalls, maxBalls)
print()

ballhalf = ceil(ballNumber/2) #Finds the middle ball for positioning

# Function for repeated action of updating string axis
def updateStringAxis():
    string[n].axis = ball[n].pos-string[n].pos # axis stretches from ball to end of string

# Creates each ball & corresponding string
for n in range(1,ballNumber+1):
    ball[n] = sphere(pos=vector(2*(n-ballhalf),0,0), radius=1) # Creates ball w/ position so that center ball is at x=0
    ball[n].m = 1 # mass
    ball[n].v = vector(0,0,0) # initial velocity
    ball[n].angmom = vector(0,0,0) # initial angular momentum
    
    string[n] = cylinder(pos=vector(ball[n].pos.x+0.001,length,0),radius = 0.1) # Creates string w/ pos.x close to ball & pos.y = length
    updateStringAxis()

sleep(0.2)

# Asks for desired number of balls to be raised & angle
ballRaiseNumber = inputNumberRange("How many balls would you like to raise?", 1, ballNumber-1)
ballRaiseAngle = inputNumberRange("What angle, in degrees, would you like to raise them to?", 10, 80)
print()

# Positions raised balls to desired angle
for n in range(1,ballRaiseNumber+1):
    theta = radians(ballRaiseAngle) # converts input angle to radians
    ball[n].pos = vector(ball[n].pos.x-length*sin(theta),(length-length*cos(theta)),0) # positions ball
    updateStringAxis()

sleep(0.2)
    
input("Here's your Newton's cradle! Press enter to start:")

g = vector(0,-9.8,0)  # acceleration due to gravity vector

dt = 0.01 # change in time for each loop
t = 0 # initial time

# Function that calculates torque & updates position for each ball
def ballCalculations():
    ball[n].theta = -atan(ball[n].pos.y / string[n].axis.x) # calculates angle using arctan
    ball[n].Fg = ball[n].m * g # calculates gravitational force = mass * gravitational acceleration
    ball[n].torque = cross(ball[n].Fg, string[n].axis) # calculates torque = cross product of gravitational force & radius
    ball[n].angmom += ball[n].torque * dt # calculates final angular momentum = initial momentum + torque * change in time

    # Calculates velocity = angular momentum / mass / radius * opposite direction of cross product of angular momentum & radius
    ball[n].v = mag(ball[n].angmom) / ball[n].m / mag(string[n].axis) * norm(-cross(ball[n].angmom,string[n].axis))

    ball[n].pos += ball[n].v * dt # calculates final position = initial position + velocity * change in time
    updateStringAxis()

# Function that transfers momentum of selected ball if a collision is detected
def momentumTransfer():
    if n < ballNumber: # Checks if selected ball is not ball on far right
        if abs(ball[n].pos.x - ball[n+1].pos.x) < 1.99986: # Checks if distance between ball & ball on right is within 2 radii
            ball[n+1].angmom = ball[n].angmom # Transfers momentum to ball on right
            ball[n].angmom = vector(0,0,0) # Sets ball momentum to 0

    if n > 1: # Checks if selected ball is not ball on far left
        if abs(ball[n].pos.x - ball[n-1].pos.x) < 1.99986: # Checks if distance between ball & ball on left is within 2 radii
            ball[n-1].angmom = ball[n].angmom # Transfers momentum to ball on left
            ball[n].angmom = vector(0,0,0) # Sets ball momentum to 0

# Runs motion simulation
while t < 50:
    rate(75) # Sets program run speed
    
    for n in range(1,ballNumber+1): # Repeats calculations & momentum transfer for each ball
        ballCalculations()
        momentumTransfer()

    t += dt # Increases time by dt